# SPARQL Queries to retrieve data from the constructed RDF graph


In [6]:
# Import necessary libraries
# !pip install rdflib # uncomment if necessary
import rdflib
from rdflib import Graph, URIRef, Namespace, SDO
import pandas as pd

# Load the graph (assuming it's saved as 'food_nutrition_graph.rdf')
BASE = Namespace("http://kg-course.io/food-nutrition/")
SCHEMA = Namespace("http://kg-course.io/food-nutrition/schema/")
g = Graph()
g.bind("base", BASE)
g.bind("schema", SCHEMA)

g.parse("KEN4256-structured-KG-Team6.ttl", format="ttl")

prefix_string = """ PREFIX ex: <http://kg-course.io/food-nutrition/schema/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <https://schema.org/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>"""

## Find recipes that are risky for individuals allergic to mango

This query retrieves recipes that contain mango as an ingredient, which could be risky for individuals with a mango allergy. It checks the recipeIngredient property for any mention of "mango" (case-insensitive) and returns the recipe name and ingredient list for those recipes.

In [19]:
results1 = g.query(
    f"""
    {prefix_string}
    SELECT * WHERE {{
        ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:recipeIngredient ?ingredientList .
        FILTER(CONTAINS(lcase(str(?ingredientList)), "mango"))
    }}
    LIMIT 10
    """
)
results1 = pd.DataFrame(results1.bindings)
results1

,ingredientList,name,recipe
0,"mango chutney, red chile, ground cumin, garlic...",Coconut Pork and Chilli Chutney,http://kg-course.io/food-nutrition/recipe/10175
1,"berries, mango, non-fat soymilk, sugar",Mixed Berry Smoothie (Vegetarian),http://kg-course.io/food-nutrition/recipe/10416
2,"mango, fresh pineapple, grapefruit juice",Tropical Sunshine Smoothie,http://kg-course.io/food-nutrition/recipe/10458
3,"tomatoes, fresh mango, garlic, zucchini, lime,...",Fresh Salsa (A La Gringo),http://kg-course.io/food-nutrition/recipe/10472
4,"boneless skinless chicken breast halves, salt,...",Chicken with Curried Mango Sauce,http://kg-course.io/food-nutrition/recipe/10501
5,"sugar, margarine, nonfat cottage cheese, low-f...",Tropical Cheesecake,http://kg-course.io/food-nutrition/recipe/10550
6,"banana, lemon juice, mango, non-fat vanilla yo...",Tropical Pina Colada Shake,http://kg-course.io/food-nutrition/recipe/10619
7,"mango, avocado, serrano chilies, red onion, ci...",Avocado-Mango Salsa,http://kg-course.io/food-nutrition/recipe/10633
8,"fresh mango, sugar, butter",Mangoes,http://kg-course.io/food-nutrition/recipe/10954
9,"garlic, tomatoes, salt, lemon juice, mango chu...",Prawns With Garlic,http://kg-course.io/food-nutrition/recipe/11169


## List all pies tagged as 'healthy' with a cooking time of less than 2 hours

This query retrieves recipes that are categorised as pies, tagged with the keyword "healthy", and have a cooking time of less than 120 minutes (2 hours). It checks the recipeCategory for "pie", the keywords for "healthy", and the cookTime for values less than 7200 seconds. The results include the recipe name, keywords, category, and cooking time, ordered by cooking time in descending order.

In [49]:
results2 = g.query(
    f"""
    {prefix_string}
    SELECT * WHERE {{
        ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:keywords ?keywords ;
            schema:recipeCategory ?recipeCategory ;
            schema:cookTime ?cookTime .

        FILTER(CONTAINS(lcase(str(?recipeCategory)), "pie") && CONTAINS(lcase(str(?keywords)), "healthy") && ?cookTime < "120"^^xsd:decimal)
    }}
    ORDER BY DESC(?cookTime)
    LIMIT 10
    """
)
results2 = pd.DataFrame(results2.bindings)
results2

,cookTime,keywords,name,recipe,recipeCategory
0,90.0,"Dessert, Apple, Fruit, Low Protein, Low Choles...",Apple Pie Filling,http://kg-course.io/food-nutrition/recipe/11166,Pie
1,82.0,"Dessert, Canadian, Low Protein, Healthy, Kid F...",Individual Pumpkin Pies,http://kg-course.io/food-nutrition/recipe/13175,Pie
2,60.0,"Dessert, Apple, Fruit, Low Protein, Low Choles...",candy apple pie,http://kg-course.io/food-nutrition/recipe/11308,Pie
3,60.0,"Dessert, Apple, Fruit, Nuts, European, Low Pro...",Apple Pecan Cobbler,http://kg-course.io/food-nutrition/recipe/13156,Pie
4,60.0,"Dessert, Healthy, Thanksgiving, Oven, < 4 Hours",Good Ol' Pumpkin Pie,http://kg-course.io/food-nutrition/recipe/2393,Pie
5,60.0,"Dessert, Fruit, Healthy, Refrigerator, < 4 Hou...",Lemon Cherry Icebox Pie,http://kg-course.io/food-nutrition/recipe/4817,Pie
6,60.0,"Dessert, Low Protein, Healthy, Free Of..., Tha...","Diabetic, Low-Fat Pumpkin Pie",http://kg-course.io/food-nutrition/recipe/5189,Pie
7,60.0,"Dessert, Low Protein, Healthy, < 15 Mins, Oven...",Rhubarb Custard Pie,http://kg-course.io/food-nutrition/recipe/8200,Pie
8,60.0,"One Dish Meal, Poultry, Meat, Low Cholesterol,...",Chicken-Vegetable Pot Pie / Pies,http://kg-course.io/food-nutrition/recipe/8342,Savory Pies
9,55.0,"Dessert, Apple, Fruit, Low Protein, Healthy, O...",Caramel Apple Pie With Crunchy Crumb Topping,http://kg-course.io/food-nutrition/recipe/3577,Pie


## List all restaurants in New Delhi that serve Chinese cuisine and offer online delivery

In [48]:
results3 = g.query(
    f"""
    {prefix_string}
    SELECT * WHERE {{
        ?restaurant a schema:Restaurant ;
            schema:name ?name ;
            schema:addressLocality ?locality ;
            schema:servesCuisine ?servesCuisine ;
            ex:hasOnlineDelivery ?deliversOnline .

        FILTER(CONTAINS(lcase(str(?locality)), "new delhi") && CONTAINS(lcase(str(?servesCuisine)), "chinese") && ?deliversOnline = "true"^^xsd:boolean)
    }}
    ORDER BY ASC(?name)
    LIMIT 10
    """
)
results3 = pd.DataFrame(results3.bindings)
results3

,deliversOnline,locality,name,recipeCategory,restaurant
0,true,"Pitampura, New Delhi",361 Restaurant & Banquet,"North Indian, Chinese, Seafood",http://kg-course.io/food-nutrition/restaurant/...
1,true,"Aggarwal City Mall, Pitampura, New Delhi",4 on 44 Restaurant & Bar,"North Indian, Chinese",http://kg-course.io/food-nutrition/restaurant/...
2,true,"Vasant Vihar, New Delhi",AOWLS,"Chinese, Fast Food",http://kg-course.io/food-nutrition/restaurant/...
3,true,"Laxmi Nagar, New Delhi",Aashirwad Restaurant,"North Indian, Chinese",http://kg-course.io/food-nutrition/restaurant/...
4,true,"Kirti Nagar, New Delhi",Above & Beyond,"North Indian, Chinese, Mughlai, Continental",http://kg-course.io/food-nutrition/restaurant/...
5,true,"R K Puram, New Delhi","Ada ""e"" Haandi's","North Indian, Mughlai, Chinese",http://kg-course.io/food-nutrition/restaurant/...
6,true,"Moti Bagh, New Delhi",Ada e Handi Restaurant,"Mughlai, North Indian, Chinese",http://kg-course.io/food-nutrition/restaurant/...
7,true,"Rajinder Nagar, New Delhi",Ada'e Handi,"North Indian, Mughlai, Chinese",http://kg-course.io/food-nutrition/restaurant/...
8,true,"Green Park, New Delhi",Adyar Ananda Bhavan,"South Indian, North Indian, Street Food, Chine...",http://kg-course.io/food-nutrition/restaurant/769
9,true,"Lajpat Nagar 2, New Delhi",Aggarwal Sweet Corner,"North Indian, Chinese, Street Food, Mithai",http://kg-course.io/food-nutrition/restaurant/...


## Find the average cost of two dining at restaurants in Davenport that serve Asian food

In [55]:
results4 = g.query(
    f"""
    {prefix_string}
    SELECT (AVG(?cost) as ?avgCostof)  WHERE {{
        ?restaurant a schema:Restaurant ;
            schema:name ?name ;
            schema:addressLocality ?locality ;
            schema:servesCuisine ?servesCuisine ;
            ex:averageCostOfTwo ?cost .

        FILTER(CONTAINS(lcase(str(?locality)), "davenport") && CONTAINS(lcase(str(?servesCuisine)), "asian"))
    }}
    ORDER BY DESC(?cost)
    """
)
results4 = pd.DataFrame(results4.bindings)
results4

,avgCost
0,25.0


## Recommend the top 5 desserts published after 2000 that are labeled as ‘Easy’ and low in calories(< 300), along with their images

In [ ]:
results5 = g.query(
    f"""
    {prefix_string}
    SELECT *  WHERE {{
        ?recipe a schema:Recipe ;
            schema:name ?name ;
            schema:keywords ?keywords ;
            schema:datePublished ?datePublished ;
            ex:hasNutrition ?nutrition .
        ?nutrition a schema:NutritionInformation ;
            schema:calories ?calories .

        #FILTER(CONTAINS(lcase(str(?keywords)), "easy") && ?calories < "300"^^xsd:decimal && ?datePublished > "2000-01-01T00:00:00Z"^^xsd:dateTime)
    }}
    ORDER BY DESC(?cost)
    LIMIT 5
    """
)
results5 = pd.DataFrame(results5.bindings)
results5

## Identify the top 10 highly rated beverages, including their preparation time and sugar content

## Identify the highest-rated recipes containing protein-rich (> 20) ingredients and check if their corresponding cuisines are commonly available in the USA restaurants

## Find the top 5 healthiest recipes based on a nutrition density score (NDS), along with their average sentiment from reviews. Check if restaurants serve cuisines linked to these recipes and retrieve their aggregate restaurant ratings.

To calculate an NDS, you will incorporate weighting factors based on dietary importance, i.e., high protein, high fiber, low sugar.

> 𝑁𝐷𝑆 = (1.0 × 𝑃𝑟𝑜𝑡𝑒𝑖𝑛(𝑔)) + (1. 5 × 𝐹𝑖𝑏𝑒𝑟(𝑔)) − (2. 0 × 𝑆𝑢𝑔𝑎𝑟(𝑔))

Protein is essential for muscle and metabolism. Fiber is crucial for digestion and satiety. Excess sugar contributes to metabolic issues, hence higher penalty